In [1]:
this_list = ["pay","cup","ship","bat","these","show","rut","down","ten","into","then","dog","bat","with","my","dig","note","cut","mom","brick"]
print(len(this_list))

20


In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [6]:
import json

# Define the file path
file_path = "src/data/word_lists.json"

# Read the JSON file
with open(file_path, "r") as file:
    data = file.read()

# Load the data into a dictionary
word_lists = json.loads(data)

# Print the number of words in each level
def word_table(word_list):
    from prettytable import PrettyTable
    table = PrettyTable()
    table.field_names = ["Level", "Words", "Unique", "Bonus", "Unique Bonus"]
    table.border = False
    table.align = "r"
    for level in word_lists["levels"]:
        words = word_lists["levels"][level]["words"]
        unique_words = set(words)
        bonus = word_lists["levels"][level]["bonus"]
        unique_bonus = set(bonus)
        table.add_row([level, len(words), len(unique_words), len(bonus), len(unique_bonus)])

    print(table)

word_table(word_lists)

 Level  Words  Unique  Bonus  Unique Bonus 
     1    250     245    168           168 
     2    250     250    168           168 
     3    250     250    168           168 
     4    250     250    168           168 
     5    250     250    168           168 
     6    250     250    168           168 
     7    250     249    168           168 
     8    250     250    168           168 
     9    250     250    168           168 
    10    250     250    168           168 
    11    250     250    168           168 
    12    504     502    168           168 
    13    350     348    168           168 


In [8]:
import json

# Define the file paths
input_file_path = "src/data/word_lists.json"
output_file_path = "src/data/word_lists_unique.json"

# Read the original JSON file
with open(input_file_path, "r") as file:
    data = file.read()

# Load the data into a dictionary
word_lists = json.loads(data)

# Create a new dictionary to store the unique words
unique_word_lists = {
    "levels": {},
    "metadata": word_lists["metadata"]  # Copy the metadata as it is
}

# Process each level to keep only unique words, ensuring they are lowercase and stripped
for level in word_lists["levels"]:
    words = word_lists["levels"][level]["words"]
    bonus_words = word_lists["levels"][level]["bonus"]
    
    # Create sets with words that are lowercased and stripped of leading/trailing whitespace
    unique_words = list(set(word.strip().lower() for word in words))
    unique_bonus_words = list(set(bonus_word.strip().lower() for bonus_word in bonus_words))
    
    unique_word_lists["levels"][level] = {
        "words": unique_words,
        "bonus": unique_bonus_words
    }

# Save the unique words dictionary to a new JSON file
with open(output_file_path, "w") as output_file:
    json.dump(unique_word_lists, output_file, indent=4)

print(f"Unique word lists have been saved to {output_file_path}")

# Reuse the word_table function to verify
def word_table(word_list):
    from prettytable import PrettyTable
    table = PrettyTable()
    table.field_names = ["Level", "Words", "Unique", "Bonus", "Unique Bonus"]
    table.border = False
    table.align = "r"
    for level in word_lists["levels"]:
        words = word_list["levels"][level]["words"]
        unique_words = set(words)
        bonus = word_list["levels"][level]["bonus"]
        unique_bonus = set(bonus)
        table.add_row([level, len(words), len(unique_words), len(bonus), len(unique_bonus)])

    print(table)

word_table(unique_word_lists)

Unique word lists have been saved to src/data/word_lists_unique.json
 Level  Words  Unique  Bonus  Unique Bonus 
     1    245     245    168           168 
     2    250     250    168           168 
     3    250     250    168           168 
     4    250     250    168           168 
     5    249     249    168           168 
     6    250     250    168           168 
     7    249     249    168           168 
     8    250     250    168           168 
     9    250     250    168           168 
    10    250     250    168           168 
    11    250     250    168           168 
    12    502     502    168           168 
    13    348     348    168           168 


In [9]:
%pip install ollama

  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached anyio-4.4.0-py3-none-any.whl (86 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model": "openhermes",
    "prompt": "Tell me a joke",
    "stream": False,
    "max_tokens": 50
}

import requests
response = requests.post(url, headers=headers, json=data)

response_text = response.json()["response"]
print(response_text)

Why don't scientists trust atoms?

Because they make up everything!


In [17]:
import requests
import json
import random

# Define the file paths
input_file_path = "src/data/word_lists_with_ollama_3.json"
output_file_path = "src/data/word_lists_with_ollama_4.json"

# Define Ollama API parameters
url = "http://localhost:11434/api/generate"
headers = {
    "Content-Type": "application/json"
}

def get_new_words(model, prompt, max_tokens=100):
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": 0.2
    }
    response = requests.post(url, headers=headers, json=data)
    response_text = response.json()["response"]
    new_words = [word.strip() for word in response_text.split(",") if word.strip()]
    new_words = ["".join(char for char in word if char.isalpha() or char == "'") for word in new_words]
    new_words = new_words[1:]
    new_words = [word.lower() for word in new_words]
    new_words = [word for word in new_words if len(word) >= 3]
    if len(new_words) >= 5:
        # print(f"Returning {min(50, len(new_words))} words")
        return list(set(new_words[:50]))  # Return exactly 10 words
    else:
        # print(f"Warning: Less than 5 words generated for prompt: '{prompt}'")
        return list(set(new_words[:50]))

# Read the original JSON file
with open(input_file_path, "r") as file:
    data = file.read()

# Load the data into a dictionary
word_lists = json.loads(data)

# get all of the unique words in all of the levels and bonus levels
all_words = []
for level in word_lists["levels"]:
    all_words.extend(word_lists["levels"][level]["words"])
    all_words.extend(word_lists["levels"][level]["bonus"])

all_words = list(set(all_words))

# Model name
model_name = "openhermes"

ollama_words = {
    "levels": {}
}  # this will mimic the dictionary structure of the word_lists
# create unique_words_so_far as a set
unique_words_so_far = set()

new_words_by_level = {}  # Temporary storage for new words by level
new_bonus_words_by_level = {}  # Temporary storage for new bonus words by level

# Iterate through each level
for level in word_lists["levels"]:
    print(f"Processing level {level}...")
    # Get words and bonus words from the original lists
    words = list(set(word_lists["levels"][level]["words"]))
    bonus_words = list(set(word_lists["levels"][level]["bonus"]))

    # Initialize the level's word and bonus word lists in ollama_words
    ollama_words["levels"][level] = {
        "words": words.copy(),
        "bonus": bonus_words.copy()
    }

    # update existing words and bonus words to the unique words so far
    unique_words_so_far.update(words)
    unique_words_so_far.update(bonus_words)
    unique_words_so_far = set(unique_words_so_far)  # Ensure uniqueness

    # Get the average word length for the level
    average_word_length = sum(len(word) for word in words) / len(words)
    average_bonus_word_length = sum(len(word) for word in bonus_words) / len(bonus_words)

    # Temporary storage for new unique words per level
    new_unique_words = []
    new_unique_bonus_words = []

    # Generate new words for the main word list
    if len(words) < 400:
        while len(new_unique_words) < 10:
            random.shuffle(words)
            prompt = f"Generate 20 new spelling words of similar length and complexity to the following list.\nSpell each correctly.\nAvoid plurals.\nProvide them in a comma-delimited format. Do not alphabetize, and do not start each with the same letter.\nLevel {level} words: {', '.join(random.sample(words, 10))}"
            generated_words = get_new_words(model_name, prompt)
            
            # Filter and select only the new words that are not already in the unique_words_so_far and are close in length to the average
            filtered_new_words = [word for word in generated_words if word not in unique_words_so_far and abs(len(word) - average_word_length) <= 2]
            
            new_unique_words.extend(filtered_new_words)
            new_unique_words = list(set(new_unique_words))  # Ensure uniqueness
            
            print(f"{len(new_unique_words)} unique new words generated for level {level} so far.")

        if new_unique_words:
            print(f"Adding {len(new_unique_words)} new words to level {level}: {new_unique_words}")
            ollama_words["levels"][level]["words"].extend(new_unique_words)
            unique_words_so_far.update(new_unique_words)  # Update only after confirming uniqueness
            new_words_by_level[level] = new_unique_words

    # Generate new words for the bonus word list
    if len(bonus_words) < 400:
        while len(new_unique_bonus_words) < 80:
            random.shuffle(bonus_words)
            prompt = f"Generate 20 new spelling words of similar length and complexity to the following list.\nSpell each correctly.\nAvoid plurals.\nProvide them in a comma-delimited format. Do not alphabetize, and do not start each with the same letter.\nLevel {level} words: {', '.join(random.sample(bonus_words, 10))}"
            generated_bonus_words = get_new_words(model_name, prompt)
            
            # Filter and select only the new words that are not already in the unique_words_so_far and are close in length to the average
            filtered_new_bonus_words = [word for word in generated_bonus_words if word not in unique_words_so_far and abs(len(word) - average_bonus_word_length) <= 2]
            
            new_unique_bonus_words.extend(filtered_new_bonus_words)
            new_unique_bonus_words = list(set(new_unique_bonus_words))  # Ensure uniqueness
            
            print(f"{len(new_unique_bonus_words)} unique new bonus words generated for level {level} so far.")

        if new_unique_bonus_words:
            print(f"Adding {len(new_unique_bonus_words)} new bonus words to level {level}: {new_unique_bonus_words}")
            ollama_words["levels"][level]["bonus"].extend(new_unique_bonus_words)
            unique_words_so_far.update(new_unique_bonus_words)  # Update only after confirming uniqueness
            new_bonus_words_by_level[level] = new_unique_bonus_words

    unique_words_so_far = set(unique_words_so_far)  # Ensure uniqueness
    print(f"{len(unique_words_so_far)} unique words so far.")

# Print the new words by level
for level in new_words_by_level:
    if len(new_words_by_level[level]):
        print(f"Level {level} new words: {', '.join(new_words_by_level[level])}")
    if len(new_bonus_words_by_level[level]):
        print(f"Level {level} new bonus words: {', '.join(new_bonus_words_by_level[level])}")

# Save the updated word lists to a new JSON file
with open(output_file_path, "w") as output_file:
    json.dump(ollama_words, output_file, indent=2)

print(f"Updated word lists with new words have been saved to {output_file_path}")

Processing level 1...
3 unique new bonus words generated for level 1 so far.
3 unique new bonus words generated for level 1 so far.
5 unique new bonus words generated for level 1 so far.
8 unique new bonus words generated for level 1 so far.
15 unique new bonus words generated for level 1 so far.
15 unique new bonus words generated for level 1 so far.
19 unique new bonus words generated for level 1 so far.
19 unique new bonus words generated for level 1 so far.
26 unique new bonus words generated for level 1 so far.
40 unique new bonus words generated for level 1 so far.
41 unique new bonus words generated for level 1 so far.
43 unique new bonus words generated for level 1 so far.
44 unique new bonus words generated for level 1 so far.
46 unique new bonus words generated for level 1 so far.
49 unique new bonus words generated for level 1 so far.
49 unique new bonus words generated for level 1 so far.
50 unique new bonus words generated for level 1 so far.
52 unique new bonus words gene

In [16]:
import json
from prettytable import PrettyTable

def word_table(word_list):
    table = PrettyTable()
    table.field_names = ["Level", "Words", "Unique", "Bonus", "Unique Bonus"]
    table.border = False
    table.align = "r"
    
    for level in word_list["levels"]:  # Use word_list here
        words = word_list["levels"][level]["words"]  # Use word_list here
        unique_words = set(words)
        bonus = word_list["levels"][level]["bonus"]  # Use word_list here
        unique_bonus = set(bonus)
        table.add_row([level, len(words), len(unique_words), len(bonus), len(unique_bonus)])

    print(table)

input_file_path = "src/data/word_lists_with_ollama_3.json"
with open(input_file_path, "r") as file:
    data = file.read()
ollama_word_lists = json.loads(data)
word_table(ollama_word_lists)

 Level  Words  Unique  Bonus  Unique Bonus 
     1    400     400    326           326 
     2    400     400    334           334 
     3    401     401    325           325 
     4    400     400    327           327 
     5    407     407    328           328 
     6    405     405    329           329 
     7    409     409    328           328 
     8    401     401    328           328 
     9    400     400    328           328 
    10    403     403    324           324 
    11    401     401    325           325 
    12    652     652    324           324 
    13    504     504    324           324 


In [ ]:
input_file_path = "src/data/word_lists_with_ollama_3.json"
output_file_path = "src/data/word_lists_with_ollama_4.json"

# Read the original JSON file
with open(input_file_path, "r") as file:
    data = file.read()

# Load the data into a dictionary
word_lists = json.loads(data)

output_data = {
    "levels": {}
}

# Process each level to keep only the first 400 words and 200 bonus words
for level in word_lists["levels"]:
    words = word_lists["levels"][level]["words"]
    bonus_words = word_lists["levels"][level]["bonus"]
    
    # Limit the words to the first 400 and bonus words to the first 200
    words = words[:400]
    bonus_words = bonus_words[:200]
    
    output_data["levels"][level] = {
        "words": words,
        "bonus": bonus_words
    }